In [3]:
# This program attempts to solve the knapsack problem using the DEAP library
# https://deap.readthedocs.io/en/master/examples/ga_knapsack.html
#https://github.com/DEAP/deap/blob/cb20d979d3b62635cc330a9804aeb29523bffd42/examples/ga/knapsack.py

import random

import numpy

!pip install deap
from deap import algorithms
from deap import base
from deap import creator
from deap import tools


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 5.6 MB/s eta 0:00:00


In [4]:
IND_INIT_SIZE = 5
MAX_ITEM = 50
MAX_WEIGHT = 50
NBR_ITEMS = 20

In [5]:
# To assure reproducibility, the RNG seed is set prior to the items
# dict initialization. It is also seeded in main().
random.seed(64)

In [6]:
# Create the item dictionary: item name is an integer, and value is 
# a (weight, value) 2-tuple.
items = {}
# Create random items and store them in the items' dictionary.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))

In [7]:
creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", set, fitness=creator.Fitness)

In [8]:
toolbox = base.Toolbox()

In [9]:
# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [10]:
def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0             # Ensure overweighted bags are dominated
    return weight, value

In [11]:
def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2

In [12]:
def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

In [13]:
toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

In [14]:
def main():
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2

    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)

    return pop, stats, hof

In [15]:
if __name__ == "__main__":
    main()    

gen	nevals	avg                        	std                      	min                      	max                        
0  	50    	[ 21.78       211.94878577]	[ 5.96419316 70.12656822]	[10.         67.91830915]	[ 38.         345.35491309]
1  	88    	[  6.98       118.10039868]	[  8.31983173 111.14178365]	[0. 0.]                  	[ 28.         345.35491309]
2  	93    	[ 1.36       32.52866072]  	[ 3.95858561 81.87027593]  	[0. 0.]                  	[ 20.         345.35491309]
3  	87    	[ 1.78       43.20812896]  	[ 4.36022935 90.48671669]  	[0. 0.]                  	[ 20.         345.35491309]
4  	93    	[ 1.84       50.38824387]  	[ 4.41524631 92.57532172]  	[0. 0.]                  	[ 20.         345.35491309]
5  	85    	[ 2.38       62.96901568]  	[  4.90668931 102.99689289]	[0. 0.]                  	[ 20.         345.35491309]
6  	82    	[ 3.12       90.34030109]  	[  5.05426553 107.34668642]	[0. 0.]                  	[ 20.         345.35491309]
7  	88    	[ 2.34       54.85219467]